In [1]:
##IMPORTS##
import chess
import numpy as np
import pandas as pd

df = pd.read_csv('chessData.csv')

def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

ModuleNotFoundError: No module named 'chess'

In [ ]:
##SYNTETIC DATA GENERATION##
def generate_synthetic_data_dict(N,size,df,seed=1):
    if size == 1:
        return generate_synthetic_data_single(N,seed,df)
    data = []
    np.random.seed(seed)
    for i in range(N):
        dict = {}
        random_list = np.random.multinomial(size, np.ones(size)/size, size=1)[0]
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        for j in range(size):
            if random_list[j] != 0:
                dict[np.random.choice(legal_moves)] = random_list[j]
        data.append(dict)

    return data

def generate_synthetic_data_single(N,seed,df):
    data = []
    np.random.seed(seed)
    for i in range(N):
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        data.append(np.random.choice(legal_moves))
    return data

path=("stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe")


def evaluate_move(board, move, path):
    #Evaluates the move before move and after move return the difference
    board1 = board.copy()
    board1.push(chess.Move.from_uci(move))
    board2 = board.copy()
    #Return differnce of board1 and board2


def amount_legal_moves_taken(l , df):
    amount = 0
    for i in range(len(l)):
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        if l[i] in legal_moves:
            amount += 1
    return amount

def amount_index_zero(l,df):
    amount = 0
    for i in range(len(l)):
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        if l[i] == legal_moves[0]:
            amount += 1
    return amount

def amount_mean_taken_over_mu_moves():
    pass


amount = amount_index_zero(generate_synthetic_data_dict(10,1,df,1),df)
amount 

1

In [ ]:
import chess
import chess.engine
import pandas as pd
import numpy as np

def evaluate_move(fen,move):
    # Path to Stockfish executable
    stockfish_path = "stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe"
    
    with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
        engine.configure({"Threads": 8})  # Increased number of threads for stability
        # Create a chess board from FEN
        board = chess.Board(fen)
        # Determine which player is to move
        is_white_to_move = board.turn == chess.WHITE
        # Evaluate the initial position
        initial_info = engine.analyse(board, chess.engine.Limit(time=0.1))  # Increased time limit for stability
        initial_score = initial_info["score"].white().score(mate_score=10000)  # Get the POV score for White
        
        # If it's Black to move, invert the score
        if not is_white_to_move:
            initial_score = -initial_score
        
        board = chess.Board(fen)
        board.push(chess.Move.from_uci(move))
        info = engine.analyse(board, chess.engine.Limit(time=0.1))  # Increased time limit for stability
        move_score_uci = info["score"].white().score(mate_score=10000)  # Get the POV score for White
        # If it's Black to move, invert the score
        if not is_white_to_move:
            move_score_uci = -move_score_uci

        move_difference = move_score_uci - initial_score
        return move_difference

def moves_dataframe(fen):   
    legal_moves = gen_legal_moves(chess.Board(fen))
        
    # Evaluate all legal moves
    move_differences = []
    for move in legal_moves:
        score = evaluate_move(fen, move)
        move_differences.append((move, score))
        
    
    # Convert to DataFrame
    moves_df = pd.DataFrame(move_differences, columns=['move', 'score_difference'])   
    return moves_df
        

def percentiles(moves_df):
        score_75th_percentile = np.percentile(moves_df['score_difference'], 75)
        score_25th_percentile = np.percentile(moves_df['score_difference'], 25)
        score_50th_percentile = np.percentile(moves_df['score_difference'], 50)
        # Return results
        result = {
            "score_75th_percentile": score_75th_percentile,
            "score_50th_percentile": score_50th_percentile,
            "score_25th_percentile": score_25th_percentile,}
        return result

def in_percentile(score,percentile):
    if score >= percentile["score_75th_percentile"]:
        return np.array([0,0,0,1])
    elif percentile["score_50th_percentile"] <= score < percentile["score_75th_percentile"]:
        return np.array([0,0,1,0])
    elif percentile["score_25th_percentile"] <= score < percentile["score_50th_percentile"]:
        return np.array([0,1,0,0])
    else:
        return np.array([1,0,0,0])
    
def percentile_distribution(moves,df):
    result = np.zeros(4)
    for i in range(len(moves)):
        moves_df = moves_dataframe(df['FEN'][i],moves[i],df)
        move_difference = evaluate_move(df['FEN'][i],moves[i])
        result += in_percentile(move_difference, percentiles(moves_df))
    return result

def stockfish_score_function(N, df):
    stockfish_score = []
    for i in range(N):
        fen = df['FEN'][i]
        moves_df = moves_dataframe(fen)
        stockfish_score.append(max(moves_df['score_difference']))
    return stockfish_score   


In [ ]:
###WASERSTEIN DISTANCE###
from scipy.stats import wasserstein_distance


llm_moves = generate_synthetic_data_dict(10,1,df)
llm_moves = [evaluate_move(df['FEN'][i],llm_moves[i]) for i in range(len(llm_moves))]
stockfish_moves = stockfish_score_function(10,df)
wasserstein_distance(llm_moves,stockfish_moves)


135.5

In [ ]:
wasserstein_distance(stockfish_moves,llm_moves)

135.5

In [ ]:
import numpy as np
from scipy.stats import wasserstein_distance

def permutation_test(sample1, sample2, num_permutations=1000):
    combined = np.concatenate([sample1, sample2])
    observed_distance = wasserstein_distance(sample1, sample2)
    
    permuted_distances = []
    for _ in range(num_permutations):
        np.random.shuffle(combined)
        permuted_sample1 = combined[:len(sample1)]
        permuted_sample2 = combined[len(sample1):]
        permuted_distances.append(wasserstein_distance(permuted_sample1, permuted_sample2))
    
    p_value = np.mean(np.array(permuted_distances) >= observed_distance)
    return observed_distance, p_value

# Example usage
sample1 = llm_moves
sample2 = stockfish_moves
observed_distance, p_value = permutation_test(sample1, sample2)
print(f"Observed Distance: {observed_distance}, p-value: {p_value}")


ModuleNotFoundError: No module named 'numpy'

In [ ]:
##DATA PREPROCESSING##

In [ ]:
##DATA ANALYSIS##